# 06 Getting a New Dataset

In [9]:
import dask, dask.dataframe as dd
import matplotlib.pyplot as plt
import pandas as pd
import re, csv, os
import numpy as np
import s3fs

pd.set_option('display.max_columns', None)
csv.field_size_limit(10000000)

%matplotlib inline

In [15]:
path = 's3://bgdata-2020/clean_data/data_clean_07/'
dir_out = 'filtered_comps_07'
partitions_out = 12

In [13]:
dtypes = {'BGTSubOcc': np.str, 'MaxExperience': np.str, 'CanonJobTitle': np.str, 'CanonIntermediary': np.str, 
          'clean_text': np.str, 'CanonYearsOfExperienceLevel': np.str, 'InternshipFlag': np.bool_, 'MaxAnnualSalary': np.float32,
          'MaxHourlySalary': np.float32, 'DivisionCode': np.str, 'CanonPostalCode': np.int16, 'CleanJobTitle': np.str,
          'CanonCounty': np.str, 'MinDegreeLevel': np.str, 'CanonJobHours': np.str, 'MinAnnualSalary': np.float32,
          'BGTOcc': np.str, 'YearsOfExperience': np.str, 'MaxDegreeLevel': np.str, 'CanonSkillClusters': np.str,
          'CanonMaximumDegree': np.str, 'CanonSkills': np.str, 'ConsolidatedDegreeLevels': np.str, 'CanonRequiredDegrees': np.str,
          'JobID': np.str, 'MinHourlySalary': np.float32, 'Longitude': np.float32, 'Latitude': np.float32,
          'CanonJobType': np.str, 'CanonState': np.str, 'LMA': np.str, 'ConsolidatedInferredNAICS': np.str,
          'MSA': np.str, 'CanonYearsOfExperienceCanonLevel': np.str, 'JobDate': np.str, 'CIPCode': np.str,
          'ConsolidatedTitle': np.str, 'ConsolidatedONET': np.str, 'MinExperience': np.str, 'EmployerClean': np.str,
          'CanonCity': np.str, 'Source': np.str, 'CanonMinimumDegree': np.str}

In [16]:
ddf = dd.read_csv(os.path.join(path, 'da*.csv'), 
                 engine='python',
                 dtype=dtypes,
                 assume_missing=True,
                 error_bad_lines=False,
                 blocksize=None,
                 usecols=dtypes.keys(),
                )
ddf

,CanonCity,CanonState,CleanJobTitle,JobDate,JobID,Latitude,Longitude,CanonPostalCode,CanonCounty,DivisionCode,LMA,MSA,CanonJobTitle,ConsolidatedONET,CanonIntermediary,InternshipFlag,Source,CanonSkillClusters,CanonSkills,CanonMaximumDegree,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MaxExperience,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,CanonJobHours,CanonJobType,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedDegreeLevels,ConsolidatedTitle,MaxDegreeLevel,MinDegreeLevel,BGTSubOcc,YearsOfExperience,EmployerClean,clean_text
npartitions=99,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,float32,float32,int16,object,object,object,object,object,object,object,bool,object,object,object,object,object,object,object,object,object,object,object,float32,float32,float32,float32,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [17]:
%%time

ddf.tail()

CPU times: user 4.08 s, sys: 904 ms, total: 4.98 s
Wall time: 7.85 s


,CanonCity,CanonState,CleanJobTitle,JobDate,JobID,Latitude,Longitude,CanonPostalCode,CanonCounty,DivisionCode,LMA,MSA,CanonJobTitle,ConsolidatedONET,CanonIntermediary,InternshipFlag,Source,CanonSkillClusters,CanonSkills,CanonMaximumDegree,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MaxExperience,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,CanonJobHours,CanonJobType,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedDegreeLevels,ConsolidatedTitle,MaxDegreeLevel,MinDegreeLevel,BGTSubOcc,YearsOfExperience,EmployerClean,clean_text
80070,Los Angeles,CA,Helicopter Mechanic,2007-12-31,339805654,34.048000,-118.291000,24470,Los Angeles,31084,DV063108|MT063110,31080: Metropolitan Statistical Area|348: Comb...,Mechanic,49302302,Unknown,False,Company from Job Board,"Maintenance, Repair, and Installation: Vehicle...","{'Aircraft Maintenance': 'Maintenance, Repair,...",Unknown,Higher Secondary Certificate,Higher Secondary Certificate,Unknown,Unknown,3,2389,49-3023.00,-9999.0,-9999.00,-9999.0,-9999.000000,fulltime,permanent,1-6,mid,12,Mechanic,Unknown,12,Automotive Service Technician / Mechanic,at least 18 months|24 months|3 year,DynCorp International,Permanent Company: DynCorp Fax Number: () Add...
80071,Anaheim,CA,Officer - Loss Prevention,2007-12-31,339846692,33.833500,-117.907997,27269,Orange,11244,DV064204|MT063110,31080: Metropolitan Statistical Area|348: Comb...,Loss Prevention Officer,33909902,Unknown,False,Company,Common Skills|Specialized Skills|Customer and ...,"{'Communication Skills': 'Common Skills', 'Com...",Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,721110,33-1099.00,-9999.0,-9999.00,-9999.0,-9999.000000,fulltime,permanent,Unknown,Unknown,Unknown,Loss Prevention Officer,Unknown,Unknown,Loss Prevention / Asset Protection Specialist,Unknown,Marriott International Incorporated,Job Posting Details Company: Job Title: Office...
80072,Lakewood,CA,Branch Account Manager,2007-12-31,339890043,33.852200,-118.135002,25178,Los Angeles,31084,DV063108|MT063110,31080: Metropolitan Statistical Area|348: Comb...,Account Manager,11303102,Unknown,False,Company from Job Board,Common Skills|Customer and Client Support: Bas...,"{'Communication Skills': 'Common Skills', 'Cus...",Unknown,Higher Secondary Certificate,Higher Secondary Certificate,Unknown,Unknown,Unknown,522110,11-3031.93,-9999.0,-9999.00,-9999.0,-9999.000000,fulltime,permanent,Unknown,Unknown,12,Account Manager,Unknown,12,Banking Branch Manager,three or more years,Citi,"Branch Account Manager Vallejo, CA #026 | Citi..."
80073,Miami,FL,Financial Services Professional,2007-12-31,360306555,25.794901,-80.127800,-32377,Miami-Dade,33124,DV123312|MT123310,33100: Metropolitan Statistical Area|370: Comb...,Unknown,41303102,Unknown,False,Job Board,Sales: Solution Sales Engineering;Specialized ...,{'Consultative Sales': 'Sales: Solution Sales ...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,6233,41-3031.00,-9999.0,-9999.00,-9999.0,-9999.000000,Unknown,Unknown,Unknown,Unknown,Unknown,Financial Services Professional,Unknown,Unknown,Financial Services Representative,Unknown,Brookdale Senior Living,- Levin Financial Group of Massmutual - Tamp...
80074,Orlando,FL,Logistics Management Specialist Documentation ...,2007-12-31,360310561,28.512800,-81.356598,-32730,Orange,Unknown,MT123674,36740: Metropolitan Statistical Area,Logistics Management Specialist,11307103,Unknown,False,Job Board,Administration: General Administrative and Cle...,{'Data Entry': 'Administration: General Admini...,Unknown,Higher Secondary Certificate,Higher Secondary Certificate,Unknown,Unknown,1,624230,11-3071.03,68787.0,33.07,52912.0,25.440001,fulltime,temporary,0-1,low,12,Logistics Management Specialist,Unknown,12,Logistics Manager,one year,Federal Emergency Management Agency,EMPLOYMENT OPPORTUNITY FEDERAL EMERGENCY MANAG...


In [18]:
df_comps_list = pd.read_csv('bg_glass_hiearchy_merge.csv', usecols=['CanonEmployer'])
df_comps_list = list(df_comps_list['CanonEmployer'].unique())
df_comps_list[:10], len(df_comps_list)

(['H5', 'Yu', 'R3', 'G5', 'Hcp', 'Square', 'Argos', 'Hgi', 'Dt', 'Cps'], 76041)

In [19]:
# %%time

comps_condition = ddf['EmployerClean'].isin(df_comps_list)
ddf0 = ddf[comps_condition]
# len(ddf), len(ddf0)

CPU times: user 402 ms, sys: 1.21 ms, total: 403 ms
Wall time: 401 ms


In [ ]:
%%time

# using the same folder in your path, we will create a new one for the cleaned data
# and save our new files there
if not os.path.exists(os.path.join(dir_out)):
    os.makedirs(os.path.join(dir_out))
    

# the following lines of code will take the last dataset, repartition it,
# and save it to the desired location. Notice the wildcard "*" below. That is
# the spot Dask will use to number your files starting from 0
(ddf0
 .repartition(npartitions=partitions_out)
 .to_csv(os.path.join(dir_out, 'data_filtered_*.csv'), index=False)
 )